In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
import pinecone
from langchain.vectorstores import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.autonotebook import tqdm

c:\Users\Admin\AppData\Local\conda\conda\envs\med\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
pip show langchain

Name: langchain
Version: 0.0.225
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: c:\users\admin\appdata\local\conda\conda\envs\med\lib\site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.


In [5]:
PINECONE_API_KEY = "4b6b83c2-dc68-4bad-ae31-9642bc3be58e"
PINECONE_API_ENV = "us-east-1"

In [8]:
# Extract the data from the pdf book
def load_data(data):
    loader = DirectoryLoader(data,
    glob="*.pdf",
    loader_cls=PyPDFLoader)

    documents = loader.load()
    return documents
    

In [41]:
pip install pyPDF

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip show pyPDF

In [3]:
extracted_pdf = load_data("data/")
print("Data Extracted")

NameError: name 'DirectoryLoader' is not defined

In [12]:
#extracted_data

In [14]:
#create text Chunks
def text_split(extracted_pdf):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                   chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_pdf)
    return text_chunks 

In [15]:
text_chunks = text_split(extracted_pdf)
print("chunks created")
print("The number of chunks is:",len(text_chunks))

chunks created
The number of chunks is: 7020


In [21]:
#download embedding model
def download_huggingfaceembedding():
    embedding = HuggingFaceEmbeddings(model_name="TheBloke/Llama-2-7B-Chat-GGML")
    return embedding

In [23]:
embedding = download_huggingfaceembedding()

PermissionError: [WinError 5] Access is denied: 'C:\\Users\\Admin/.cache\\torch'

In [15]:
embedding

NameError: name 'embedding' is not defined

In [27]:
query_result = embedding.embed_query("Hello World!")
print("Length:", len(query_result))
query_result

NameError: name 'embedding' is not defined

In [31]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

AttributeError: module 'langchain.vectorstores.pinecone' has no attribute 'init'

In [ ]:
# After having my index, we can load it like this - 
docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embedding)

query = "What are Allegies?"

docs = docsearch.similarity_search(query=query, k=3)

print("Result:", docs)

In [ ]:
prompt_template = """
Use the following piece of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to generate any random answer from your own

Context:{context}
Question:{question}

Only return the helpful answer and nothing else
helpful answer:
"""

In [ ]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
chain_type_kwargs = {"prompt":PROMPT}

In [ ]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={"max_new_tokens":512,
                            'temperature':0.8})

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever= docsearch.as_retriever(search_kwargs={"k":2}),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input = input(f"Input Prompt: ")
    result = qa({"query":user_input})
    print("Response:", result["result"])